In [7]:
!pip install streamlit pyngrok plotly pandas scikit-learn


In [8]:
ngrok.kill()

In [17]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px
from sklearn.ensemble import IsolationForest
from datetime import datetime

st.set_page_config(page_title="FitPulse Health Dashboard", layout="wide")

st.title("🏃 FitPulse – Health Anomaly Detection Dashboard")

# ----------------------------
# Upload Section
# ----------------------------
uploaded_file = st.file_uploader("Upload Fitness Data (CSV / JSON)", type=["csv", "json"])

@st.cache_data
def load_data(file):
    if file.name.endswith(".csv"):
        df = pd.read_csv(file)
    else:
        df = pd.read_json(file)

    # Convert all column names to lowercase to handle case insensitivity
    df.columns = df.columns.str.lower()

    if 'timestamp' not in df.columns:
        st.error(f"Error: The uploaded file is missing the 'timestamp' column. Found columns: {', '.join(df.columns)}")
        st.stop() # Stop execution if critical column is missing

    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values("timestamp")
    return df

# ----------------------------
# Anomaly Detection
# ----------------------------
def detect_anomalies(df, column):
    model = IsolationForest(contamination=0.05, random_state=42)
    df[f"{column}_anomaly"] = model.fit_predict(df[[column]])
    df[f"{column}_anomaly"] = df[f"{column}_anomaly"].map({1: 0, -1: 1})
    return df

# ----------------------------
# Main Logic
# ----------------------------
if uploaded_file:
    df = load_data(uploaded_file)

    st.success("✅ Data loaded successfully")

    # Sidebar Filters
    st.sidebar.header("🔎 Filters")

    min_date = df['timestamp'].min().date()
    max_date = df['timestamp'].max().date()

    date_range = st.sidebar.date_input(
        "Select Date Range",
        [min_date, max_date]
    )

    # Updated metric options to match lowercased column names
    metric = st.sidebar.selectbox(
        "Select Metric",
        ["heart_rate", "sleep_duration", "daily_steps"]
    )

    mask = (df['timestamp'].dt.date >= date_range[0]) & (df['timestamp'].dt.date <= date_range[1])
    df_filtered = df.loc[mask]

    df_filtered = detect_anomalies(df_filtered, metric)

    # ----------------------------
    # Visualization
    # ----------------------------
    st.subheader(f"📈 {metric.replace('_',' ').title()} Trend with Anomalies")

    fig = px.line(
        df_filtered,
        x="timestamp",
        y=metric,
        title=f"{metric.replace('_',' ').title()} Over Time"
    )

    anomalies = df_filtered[df_filtered[f"{metric}_anomaly"] == 1]

    fig.add_scatter(
        x=anomalies["timestamp"],
        y=anomalies[metric],
        mode="markers",
        marker=dict(color="red", size=8),
        name="Anomaly"
    )

    st.plotly_chart(fig, use_container_width=True)

    # ----------------------------
    # Report Generation
    # ----------------------------
    st.subheader("📄 Anomaly Report")

    report = df_filtered[df_filtered[f"{metric}_anomaly"] == 1][
        ["timestamp", metric]
    ]

    report["metric"] = metric
    report.rename(columns={metric: "value"}, inplace=True)

    st.dataframe(report)

    csv = report.to_csv(index=False).encode("utf-8")

    st.download_button(
        label="⬇ Download Anomaly Report (CSV)",
        data=csv,
        file_name="anomaly_report.csv",
        mime="text/csv"
    )

else:
    st.info("⬆ Please upload a fitness data file to continue")

Overwriting app.py


In [18]:
from pyngrok import ngrok
import subprocess
import time
import os

ngrok.set_auth_token("369parnaySCv15gtHG6XN3DD3O7_7ApDMPnUi1rPdRKRzQtTG")

# Kill any existing ngrok tunnels using pyngrok's kill function
ngrok.kill()

# Additionally, try to kill any lingering ngrok processes at the OS level
try:
    subprocess.run(["killall", "ngrok"], check=True, capture_output=True)
    print("Killed any lingering ngrok processes.")
except subprocess.CalledProcessError as e:
    print(f"No active ngrok processes found to kill, or an error occurred: {e.stderr.decode().strip()}")
except FileNotFoundError:
    print("killall command not found. This might be expected in some environments.")

# Ensure no other Streamlit process is running on 8501
try:
    # `fuser -k 8501/tcp` kills processes using port 8501
    subprocess.run(["fuser", "-k", "8501/tcp"], check=False, capture_output=True)
    print("Attempted to kill any process using port 8501.")
except FileNotFoundError:
    print("fuser command not found. This might be expected in some environments.")


# Start Streamlit in the background as a detached process
streamlit_command = [
    "streamlit", "run", "app.py",
    "--server.port", "8501",
    "--server.enableCORS=false",
    "--server.enableXsrfProtection=false"
]
streamlit_log_file = "streamlit_output.log"

print(f"Starting Streamlit app in background, output redirected to {streamlit_log_file}")
with open(streamlit_log_file, "w") as f_log:
    streamlit_process = subprocess.Popen(
        streamlit_command,
        stdout=f_log,
        stderr=subprocess.STDOUT, # Redirect stderr to stdout to capture all output in one file
        preexec_fn=os.setsid,      # Detach from the controlling terminal
        close_fds=True             # Close file descriptors in child process
    )
print(f"Streamlit app started in the background with PID: {streamlit_process.pid}. Output logged to {streamlit_log_file}")

# Give Streamlit some time to initialize
time.sleep(7) # Increased sleep time to ensure Streamlit starts fully

# Now connect ngrok to the Streamlit app
public_url = ngrok.connect(8501)
print("Streamlit URL:", public_url)


No active ngrok processes found to kill, or an error occurred: ngrok: no process found
Attempted to kill any process using port 8501.
Starting Streamlit app in background, output redirected to streamlit_output.log
Streamlit app started in the background with PID: 8041. Output logged to streamlit_output.log
Streamlit URL: NgrokTunnel: "https://lucian-volitional-implosively.ngrok-free.dev" -> "http://localhost:8501"
